In [17]:
import os
import sys
from collections import defaultdict, Counter
import time
from matplotlib import pyplot

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util
from abxportinf.busdef import BusDef
from abxportinf._optimize import MatchCost

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
block_root = '/ddr'
comp_json_path = os.path.join(block_root, 'ddr.json5')
ports = abxportinf.get_ports_from_json5(comp_json_path)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed


In [5]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [50]:
stime = time.time()
pg_bus_mappings = abxportinf.get_bus_matches(ports, list(bus_defs))
etime = time.time()
print('total time: {}s'.format(etime-stime))

initial port groups matched 22
opt port groups matched 14
total time: 66.32354712486267s


In [68]:
dport = ('axi1_ARPROT', 2, 1)
#dport = ('axi0_AWPROT', 2, 1)
#dport = ('axi0_AWQOS', 1, 1)
for port_group, bus_mappings in sorted(
    pg_bus_mappings,
    key=lambda x: x[1][0][0],
):
    lcost = bus_mappings[0][0]
    print(len(port_group), lcost)
    if lcost.dc < 2:
        print(port_group)
    #print(port_group)
    #if dport in port_group:
    #    #print('port group', len(port_group), lcost)
    #    #print(port_group)
    #    abxportinf.debug_bus_mapping(port_group, bus_mappings[0])
    #    print('size of port_group match', len(port_group))
    #    print('  - lcost', lcost)

22 12.69(n:1.69;w:11;d:0)
{('dfi_bank_p1', 3, -1), ('dfi_phyupd_type', 2, 1), ('dfi_bg_p1', 2, -1), ('dfi_lp_ack', 1, 1), ('dfi_wrlvl_en', 9, -1), ('dfi_phyupd_ack', 1, -1), ('dfi_init_complete', 1, 1), ('dfi_wrlvl_strobe', 9, -1), ('dfi_lp_req', 1, -1), ('dfi_rdlvl_req', 9, 1), ('dfi_odt_p1', 2, -1), ('dfi_wrdata_p1', 144, -1), ('dfi_rdlvl_resp', 9, 1), ('dfi_wrlvl_resp', 9, 1), ('dfi_cke_p1', 2, -1), ('dfi_address_p1', 17, -1), ('dfi_ctrlupd_ack', 1, 1), ('dfi_lvl_periodic', 9, -1), ('dfi_phyupd_req', 1, 1), ('dfi_wrlvl_req', 9, 1), ('dfi_ctrlupd_req', 1, -1), ('dfi_lp_wakeup', 4, -1)}
21 13.72(n:1.72;w:12;d:0)
{('axi1_RRESP', 2, -1), ('axi1_BPARITY', 1, -1), ('axi1_BVALID', 1, -1), ('axi1_RREADY', 1, 1), ('axi1_RVALID', 1, -1), ('axi1_ARBURST', 2, 1), ('axi1_BRESP', 2, -1), ('axi1_ARSIZE', 3, 1), ('axi1_ARLOCK', 1, 1), ('axi1_BID', 12, -1), ('axi1_ARPROT', 2, 1), ('axi1_WREADY', 1, -1), ('axi1_ARLEN', 8, 1), ('axi1_ARVALID', 1, 1), ('axi1_RID', 12, -1), ('axi1_AWREADY', 1, -1), ('ax

In [74]:
print(len(ports))
for i, (lcost, port_group, bus_mappings) in enumerate(pg_bus_mappings):
    #if i != 20:
    #    continue
    print('port_group', i)
    print('  - port_group size', len(port_group))
    for j, bus_mapping in enumerate(bus_mappings):
        cost, fcost, mapping, match_cost_func, bus_def = bus_mapping
        print('  - cost {}, fcost {}, bus_abstract {}'.format(
            cost,
            fcost,
            bus_def.abstract_type
        ))
        
    #best_mapping = bus_mappings[0]
    #abxportinf.debug_bus_mapping(port_group, best_mapping)
    
    #if lcost.dc != 0:
    #    continue
    #best_mapping = bus_mappings[0]
    #abxportinf.debug_bus_mapping(port_group, best_mapping)

296


TypeError: set_defaults() missing 1 required positional argument: 'parent'

In [66]:
#%load_ext line_profiler

dbus_def = bus_defs[0]
print('port group size', len(ports))
print('bus def num req', dbus_def.num_req_ports)
print('bus def num opt', dbus_def.num_opt_ports)

stime = time.time()
res = abxportinf._optimize.map_ports_to_bus(ports, dbus_def)
etime = time.time()
print('total time:', etime-stime)

_, mapping, match_cost_func = res
for cost, (pp, bp) in sorted(map(
    lambda x: (match_cost_func(x[0], x[1]), (x[0], x[1])),
    mapping.items()
)):
    print('  - cost:{}    {}:{}'.format(cost, pp, bp))
#%lprun -f map_ports_to_bus map_ports_to_bus(dport_group, dbus_def)

port group size 296
bus def num req 9
bus def num opt 13
total time: 11.052737474441528
  - cost:4(n:4;w:False;d:False)    ('clk', 1, 1):('HCLK', 1, 1)
  - cost:4(n:4;w:False;d:False)    ('phy_reg_command_ready', 1, 1):('HREADY', 1, 1)
  - cost:4(n:4;w:False;d:False)    ('phy_reg_write', 1, -1):('HWRITE', 1, -1)
  - cost:5(n:4;w:True;d:False)    ('dfi_wrlvl_resp', 9, 1):('HRESP', None, 1)
  - cost:5(n:4;w:True;d:False)    ('phy_reg_addr', None, -1):('HADDR', 32, -1)
  - cost:5(n:5;w:False;d:False)    ('axi0_ARESETn', 1, 1):('HRESETn', 1, 1)
  - cost:6(n:5;w:True;d:False)    ('dfi_rddata_cs_n_0', 2, -1):('HWDATA', None, -1)
  - cost:6(n:4;w:True;d:True)    ('dfi_wrdata_cs_n_0', 2, -1):('HRDATA', None, 1)
  - cost:6(n:6;w:False;d:False)    ('dfi_phyupd_ack', 1, -1):('HLOCK', 1, -1)
  - cost:7(n:6;w:True;d:False)    ('dll_rst_n', 1, -1):('HBURST', 3, -1)
  - cost:7(n:7;w:False;d:False)    ('dfi_bank_p1', 3, -1):('HSIZE', 3, -1)
  - cost:7(n:6;w:True;d:False)    ('dfi_ras_n_p1', 1, -1):('H

In [44]:
_, mapping, match_cost_func = res
for cost, (pp, bp) in sorted(map(
    lambda x: (match_cost_func(x[0], x[1]), (x[0], x[1])),
    mapping.items()
)):
    print('  - cost:{}    {}:{}'.format(cost, pp, bp))

  - cost:4(n:4;w:False;d:False)    ('clk', 1, 1):('HCLK', 1, 1)
  - cost:4(n:4;w:False;d:False)    ('phy_reg_command_ready', 1, 1):('HREADY', 1, 1)
  - cost:4(n:4;w:False;d:False)    ('phy_reg_write', 1, -1):('HWRITE', 1, -1)
  - cost:5(n:4;w:True;d:False)    ('dfi_wrlvl_resp', 9, 1):('HRESP', None, 1)
  - cost:5(n:4;w:True;d:False)    ('phy_reg_addr', None, -1):('HADDR', 32, -1)
  - cost:5(n:5;w:False;d:False)    ('axi0_ARESETn', 1, 1):('HRESETn', 1, 1)
  - cost:6(n:5;w:True;d:False)    ('dfi_rddata_cs_n_0', 2, -1):('HWDATA', None, -1)
  - cost:6(n:4;w:True;d:True)    ('dfi_wrdata_cs_n_0', 2, -1):('HRDATA', None, 1)
  - cost:6(n:6;w:False;d:False)    ('dfi_phyupd_ack', 1, -1):('HLOCK', 1, -1)
  - cost:7(n:6;w:True;d:False)    ('dll_rst_n', 1, -1):('HBURST', 3, -1)
  - cost:7(n:7;w:False;d:False)    ('dfi_bank_p1', 3, -1):('HSIZE', 3, -1)
  - cost:7(n:6;w:True;d:False)    ('dfi_ras_n_p1', 1, -1):('HTRANS', 2, -1)
  - cost:7(n:7;w:False;d:False)    ('dfi_ctrlupd_ack', 1, 1):('HCLKEN', 1